# Posit Arithmetic Tests

<div>
<img src="7_1_posits.png" width="50%"/>
</div>

#### Test 1: unchecked positive +
a + b = c

In [5]:
from pyposit import posit

def basic_addition(a_: float = 8.0, b_: float = 4.0, es = 1, n_ = 7):
    print("start")
    print(f"find {a_} + {b_}")
    a = posit(es, "0110100")
    a.from_float(a_, n_, es)
    b = posit(es, "1010000")
    b.from_float(b_, n_, es)
    print(f"A: {a}, B: {b}")
    print(f"Approx {a.to_float()} + {b.to_float()}")

    ## extract signs
    a_s = a.sign_i()
    b_s = b.sign_i()

    ## extract fraction numeric values
    a_f_s = a.frac_str()
    b_f_s = b.frac_str()
    a_f = 0
    b_f = 0
    if a.frac_len():
        a_f = 2**(-a.frac_len())*int(a.frac_str(), 2)
    if b.frac_len():
        b_f = 2**(-b.frac_len())*int(b.frac_str(), 2)

    ## extract exponent numeric values
    a_e = 0
    b_e = 0
    if a.exp_len():
        a_e = int(a.exp_str(), 2)
    if b.exp_len():
        b_e = int(b.exp_str(), 2)

    ## extract regime numeric values
    if a.rbar_str() == "1":
        a_r = -1*a.regime_len()
    else:
        a_r = a.regime_len() - 1
    if b.rbar_str() == "1":
        b_r = -1*b.regime_len()
    else:
        b_r = b.regime_len() - 1

    print(f"A~ S:{a_s} R:{a_r} E:{a_e} F:{a_f}")
    print(f"B~ S:{b_s} R:{b_r} E:{b_e} F:{b_f}")

    ## Check which is bigger and shift to match
    ## if they are equal it doesn't really matter as shift will be 0

    sf = 2**(es) # es
    frac_smol = ""
    frac_big = ""
    e_out = -1 # pre-shift the p1
    r_out = 0
    
    # a >= b
    if (a_r > b_r) or \
       (a_r == b_r and a_e > b_e) or \
       (a_r == b_r and a_e == b_e and a_f >= b_f):
        ## a is the larger
        print("A is larger")
        exp_adj = sf*(a_r-b_r) + (a_e-b_e)
        # use its base exponents
        e_out += a_e
        r_out += a_r
        ## shift the smaller numbers fraction so that it matches 
        ## add the extra 1 thats hidden (we will do this for big aswell)
        frac_smol = "1" + b_f_s
        frac_smol = "0"*(exp_adj) + frac_smol
        frac_big = "1" + a_f_s
    else:
        ## b is the larger
        print("B is larger")
        exp_adj = sf*(b_r-a_r) + (b_e-a_e)
        e_out += b_e
        r_out += b_r
        ## shift the smaller numbers fraction so that it matches 
        ## add the extra 1 thats hidden (we will do this for big aswell)
        frac_smol = "1" + a_f_s
        frac_smol = "0"*(exp_adj) + frac_smol
        frac_big = "1" + b_f_s

    # extend big to match depth
    frac_big = frac_big + (len(frac_smol) - len(frac_big))*"0"

    print(f"big:   0.{frac_big}")
    print(f"small: 0.{frac_smol}")

    ## peform the fractional addition
    ## add and zero extend
    f_sum = bin(int(frac_big, 2) + int(frac_smol, 2))[2:]
    print(f"Sum: {frac_big} + {frac_smol} = {f_sum}")

    if len(f_sum) > len(frac_smol):
        ## overflow case
        #raise BaseException("Fraction Summation Overflow")
        e_out += 1
    elif len(f_sum) <= len(frac_smol):
        f_sum = (max(len(frac_smol), len(frac_big)) - len(f_sum))*"0" + f_sum

    print(f"Sum: 0.{f_sum}")

    ## normalise the sum (i.e. reshift until the first 1 is gone)
    c = f_sum.find("1") + 1
    f_sum = f_sum[c:]
    print(f"C Normalised by {c}: 1.{f_sum}")
    ## update the exponent with the normalised shift
    e_out = e_out + c
    ## recompare the regime and the exponent levels
    ## 2^e8 vs 2^2^es^r8
    print(f"Prenormal: R:{r_out} E:{e_out} F:(1).{f_sum}")
    if (e_out > 2**es):
        e_out -= 2**es
        e_out += 1
    print(f"Posnormal: R:{r_out} E:{e_out} F:(1).{f_sum}")

    ## we now have all the ideal required parts, convert to closest posit repr by available space
    ## TODO: rounding probably invalid for frac len ~ 0 region
    ## calculate lengths required by each field
    n = n_
    sign_ = 1
    ## if 𝑅0 = 0, then 𝑟 = −𝑘.
    ## if 𝑅0 = 1, then 𝑟 = 𝑘 − 1
    regime_ = -r_out if r_out < 0 else r_out + 1
    rnought_ = 1
    exponent_ = 1
    if sign_ + regime_ >= n:
        print("Infinity")
    elif sign_ + regime_ + rnought_ == n:
        #  s r rbar 
        f_sum = ""
        e_out = 0 ## not really necessary
    elif sign_ + regime_ + rnought_ + exponent_ == n:
        #  s r rbar e
        # no frac
        f_sum = ""
    else:
        ## we have some frac space - follow the rounding standard

        # Let 𝑢 and 𝑤 be 𝑛-bit posit values such that the open interval (𝑢, 𝑤) contains 𝑥 but no 𝑛-bit posit value.
        # Let 𝑈 be the 𝑛-bit representation of 𝑢.
        # Let 𝑣 be the (𝑛 + 1)-bit posit value associated with the (𝑛 + 1)-bit representation 𝑈1.
        # if 𝑢 < 𝑥 < 𝑣 or (𝑥 = 𝑣 and LSB of 𝑈 is 0) then
        # return 𝑢
        # else
        # return 𝑤

        ## absolute allowed fraction length
        f_depth = n - (sign_ + regime_ + rnought_ + exponent_)
        
        ## remove trailing zeroes 
        f_sum = f_sum[:f_sum.rfind("1")+1]
        ## extend if necessary to required depth
        f_sum = f_sum + "0"*(f_depth- len(f_sum))

        print(f"Length : {len(f_sum)} Allowed : {f_depth} -> (1).{f_sum[:f_depth]}({f_sum[f_depth:]})")
        
        if (len(f_sum) == f_depth):
            ## then we are exact
            print("Repr Exact")
            pass
        else:
            ## we need to round
            ## round down case
            if f_sum[f_depth-1] == "0" or f_sum[f_depth-1] == "1" and f_sum[f_depth] == "0":
                print(f"Round Down {f_sum} {f_depth}")
                print(f"Reasons: {f_sum[f_depth-1]},{f_sum[f_depth]} = '0X' or '10'")
                f_sum = f_sum[:f_depth]
            ## round up case
            else:
                print("Round Up")
                print(f"Reasons: {f_sum[f_depth-1]},{f_sum[f_depth]} != '0X' or '10'")
                ## shift right to get MSB
                f_sum = "1" + f_sum
                ## add one
                roundup = bin(int(f_sum, 2) + int("1", 2))[2:]
                ## check for overflow
                if len(roundup) > len(f_sum):
                    raise BaseException("Rounding Overflow Not Implemented")
                
                ## shift left one and round up
                f_sum = roundup[1: f_depth+1]



    print(f"Poscat: R:{r_out} E:{e_out} F:(1).{f_sum}")
    f = 2**(-len(f_sum))*int(f_sum, 2)
    sff = 2**(2**(es))

    print(f"Out: {1+f} * { 2**(e_out)} * {sff**(r_out)}")
    print( (1 + f) * 2**(e_out) * sff**(r_out) )

    

In [6]:
basic_addition(100, 1, 2, 16)
print("")
print(7/32,3/16,5/32, 1/8)
print(1, 9/8, 5/4)

start
find 100 + 1
A: 0110101001000000, B: 0100000000000000
Approx 100.0 + 1.0
A~ S:1 R:1 E:2 F:0.5625
B~ S:1 R:0 E:0 F:0.0
A is larger
big:   0.110010000000000000
small: 0.000000100000000000
Sum: 110010000000000000 + 000000100000000000 = 110010100000000000
Sum: 0.110010100000000000
C Normalised by 1: 1.10010100000000000
Prenormal: R:1 E:2 F:(1).10010100000000000
Posnormal: R:1 E:2 F:(1).10010100000000000
Length : 11 Allowed : 11 -> (1).10010100000()
Repr Exact
Poscat: R:1 E:2 F:(1).10010100000
Out: 1.578125 * 4 * 16
101.0

0.21875 0.1875 0.15625 0.125
1 1.125 1.25


In [7]:
print(0.1875-0.125, 0.25-0.1875)

0.0625 0.0625


In [8]:
print(0.24)
x = posit(1, "0001110") ## 7-1
x.from_float(7, 7, 1)
print(x, x.to_float_2c())

0.24
0001111 0.21875


In [11]:
a = posit(1, "0001110") ## 7-1
a.from_float(7, 7, 1)
b = posit(1, "0001110") ## 7-1
b.from_float(-1, 7, 1)
print(a+b)

A: 0110011, B: 1100000
Approx 7.0 + -1.0
A~ S:1 R:1 E:0 F:0.75
B~ S:-1 R:0 E:0 F:0.0
A is larger
big:   0.111000
small: 0.001000
Sum: 111000 + 001000 = 1000000
Sum: 0.1000000
C Normalised by 1: 1.000000
Prenormal: R:1 E:1 F:(1).000000
Posnormal: R:1 E:1 F:(1).000000
Length : 2 Allowed : 2 -> (1).00()
Repr Exact
Poscat: R:1 E:1 F:(1).00
Out: 1.0 * 2 * 4
8.0
0 + R2*X + R_Y + E1 + F00
0110100
0110100


In [12]:
basic_addition(7, -1)

start
find 7 + -1
A: 0110011, B: 1100000
Approx 7.0 + -1.0
A~ S:1 R:1 E:0 F:0.75
B~ S:-1 R:0 E:0 F:0.0
A is larger
big:   0.111000
small: 0.001000
Sum: 111000 + 001000 = 1000000
Sum: 0.1000000
C Normalised by 1: 1.000000
Prenormal: R:1 E:1 F:(1).000000
Posnormal: R:1 E:1 F:(1).000000
Length : 2 Allowed : 2 -> (1).00()
Repr Exact
Poscat: R:1 E:1 F:(1).00
Out: 1.0 * 2 * 4
8.0
